In [1]:
import os
import cv2
import numpy as np
import torch
import json
from tqdm.notebook import tqdm  # Use tqdm.notebook for nicer progress bars in Jupyter
import shutil

# ==========================================
# 1. CONFIGURATION (Strictly from Thesis PDF)
# ==========================================
# Input: The Unified folder you organized
SOURCE_DIR = r"C:\UCF_video_dataset\Raw_Videos_Unified"

# Output: Where the clips will be saved
OUTPUT_DIR = r"C:\UCF_video_dataset\Processed_Clips"

# LOGIC PARAMETERS
SEQ_LEN = 16        # Frames per clip (Depth)
STRIDE = 5          # Dilation (Skip 4, take 5th)
CLIP_STEP = 64      # Sliding Window Jump
IMG_SIZE = 224      # Resize for TimeSformer

print("✅ Configuration Loaded.")
print(f"   Input: {SOURCE_DIR}")
print(f"   Output: {OUTPUT_DIR}")
print(f"   Logic: Seq={SEQ_LEN}, Stride={STRIDE}, Step={CLIP_STEP}")

✅ Configuration Loaded.
   Input: C:\UCF_video_dataset\Raw_Videos_Unified
   Output: C:\UCF_video_dataset\Processed_Clips
   Logic: Seq=16, Stride=5, Step=64


In [2]:
def check_system():
    print("="*50)
    print("SYSTEM DIAGNOSTICS")
    print("="*50)
    
    # Check GPU
    if torch.cuda.is_available():
        gpu_name = torch.cuda.get_device_name(0)
        print(f"✅ GPU DETECTED: {gpu_name}")
        print("   Status: Ready for Phase 2 (TimeSformer).")
        print("   Note: Phase 1 (Frame Extraction) will use CPU/Disk mainly.")
    else:
        print("⚠️ GPU NOT DETECTED!")
        print("   Phase 2 will be very slow. Check your CUDA installation.")
        
    # Check Source Directory
    if not os.path.exists(SOURCE_DIR):
        print(f"❌ ERROR: Source directory not found: {SOURCE_DIR}")
        return False
    
    # Check classes
    try:
        classes = [d for d in os.listdir(SOURCE_DIR) if os.path.isdir(os.path.join(SOURCE_DIR, d))]
        print(f"✅ DATASET FOUND: {len(classes)} Classes detected.")
        print(f"   Classes: {classes}")
        return True
    except Exception as e:
        print(f"❌ ERROR accessing source directory: {e}")
        return False

# Run the check
system_ready = check_system()

SYSTEM DIAGNOSTICS
✅ GPU DETECTED: NVIDIA GeForce RTX 3080 Ti
   Status: Ready for Phase 2 (TimeSformer).
   Note: Phase 1 (Frame Extraction) will use CPU/Disk mainly.
✅ DATASET FOUND: 15 Classes detected.
   Classes: ['Abuse', 'Arrest', 'Arson', 'Assault', 'Burglary', 'Explosion', 'Fighting', 'Normal', 'RoadAccidents', 'Robbery', 'Shooting', 'Shoplifting', 'Splits', 'Stealing', 'Vandalism']


In [3]:
def process_video_sliding_window(video_path, save_root_dir):
    """
    Splits video into multiple overlapping clips (Bags of Instances).
    Returns the number of clips created.
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        return 0

    # Read all frames into memory (Fastest for 3080Ti systems with high RAM)
    frames_buffer = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        # Resize immediately to save RAM and Disk space
        frame = cv2.resize(frame, (IMG_SIZE, IMG_SIZE))
        frames_buffer.append(frame)
    cap.release()
    
    # Safety check: Video too short?
    min_needed = (SEQ_LEN - 1) * STRIDE + 1
    if len(frames_buffer) < min_needed:
        return 0

    # --- THE CRITICAL FIX: SLIDING WINDOW LOOP ---
    clip_count = 0
    
    # Start at 0, slide forward by CLIP_STEP (64)
    for start_idx in range(0, len(frames_buffer), CLIP_STEP):
        
        # Calculate indices: [start, start+5, start+10, ... up to 16 frames]
        indices = [start_idx + (i * STRIDE) for i in range(SEQ_LEN)]
        
        # Check if the last frame needed is inside the video
        if indices[-1] >= len(frames_buffer):
            break
            
        # Create the Sub-Folder for this Clip
        # Example: .../Explosion001/clip_0000/
        clip_name = f"clip_{clip_count:04d}"
        clip_dir = os.path.join(save_root_dir, clip_name)
        os.makedirs(clip_dir, exist_ok=True)
        
        # Save the 16 images
        for i, frame_idx in enumerate(indices):
            img = frames_buffer[frame_idx]
            save_path = os.path.join(clip_dir, f"img_{i:03d}.jpg")
            cv2.imwrite(save_path, img)
            
        clip_count += 1
        
    return clip_count

In [4]:
import json # Make sure this is imported

def main():
    if not check_system():
        return

    print("\n" + "="*50)
    print("STARTING PHASE 1: SLIDING WINDOW EXTRACTION (WITH METADATA)")
    print("="*50)

    classes = os.listdir(SOURCE_DIR)
    
    # --- METADATA STORAGE ---
    # This list will track every video we process
    dataset_metadata = [] 

    for class_name in classes:
        class_path = os.path.join(SOURCE_DIR, class_name)
        if not os.path.isdir(class_path):
            continue
            
        # Create Class Output Directory
        class_out_dir = os.path.join(OUTPUT_DIR, class_name)
        os.makedirs(class_out_dir, exist_ok=True)
        
        # Get list of videos
        videos = [f for f in os.listdir(class_path) if f.lower().endswith(('.mp4', '.avi', '.mkv'))]
        
        print(f"\n📂 Processing Class: {class_name} ({len(videos)} videos)")
        
        for vid_file in tqdm(videos, desc=f"Extracting {class_name}", unit="vid"):
            
            video_path = os.path.join(class_path, vid_file)
            video_name = os.path.splitext(vid_file)[0]
            
            # Output path: .../Processed_Clips/Explosion/Explosion001
            video_out_dir = os.path.join(class_out_dir, video_name)
            
            # 1. RUN EXTRACTION
            n_clips = 0
            
            # Check if already processed to save time
            if os.path.exists(video_out_dir) and len(os.listdir(video_out_dir)) > 0:
                n_clips = len(os.listdir(video_out_dir)) # Count existing folders
            else:
                try:
                    n_clips = process_video_sliding_window(video_path, video_out_dir)
                    if n_clips == 0 and os.path.exists(video_out_dir):
                        shutil.rmtree(video_out_dir) # Delete empty folders
                except Exception as e:
                    print(f"❌ Error extracting {vid_file}: {e}")
                    continue

            # 2. LOG METADATA (Crucial for Phase 2)
            if n_clips > 0:
                record = {
                    "video_name": video_name,
                    "class_name": class_name,
                    "num_clips": n_clips,
                    "clips_path": video_out_dir,
                    "original_video_path": video_path
                }
                dataset_metadata.append(record)

    # --- SAVE METADATA FILE ---
    metadata_path = os.path.join(OUTPUT_DIR, "dataset_metadata.json")
    with open(metadata_path, 'w') as f:
        json.dump(dataset_metadata, f, indent=4)

    print("\n" + "="*50)
    print("✅ PHASE 1 COMPLETE")
    print(f"📄 Metadata saved to: {metadata_path}")
    print(f"📊 Total Videos Processed: {len(dataset_metadata)}")
    print("Next Step: Load 'dataset_metadata.json' in Phase 2.")
    print("="*50)

if __name__ == "__main__":
    main()

SYSTEM DIAGNOSTICS
✅ GPU DETECTED: NVIDIA GeForce RTX 3080 Ti
   Status: Ready for Phase 2 (TimeSformer).
   Note: Phase 1 (Frame Extraction) will use CPU/Disk mainly.
✅ DATASET FOUND: 15 Classes detected.
   Classes: ['Abuse', 'Arrest', 'Arson', 'Assault', 'Burglary', 'Explosion', 'Fighting', 'Normal', 'RoadAccidents', 'Robbery', 'Shooting', 'Shoplifting', 'Splits', 'Stealing', 'Vandalism']

STARTING PHASE 1: SLIDING WINDOW EXTRACTION (WITH METADATA)

📂 Processing Class: Abuse (50 videos)


Extracting Abuse:   0%|          | 0/50 [00:00<?, ?vid/s]


📂 Processing Class: Arrest (50 videos)


Extracting Arrest:   0%|          | 0/50 [00:00<?, ?vid/s]


📂 Processing Class: Arson (50 videos)


...


📂 Processing Class: Stealing (100 videos)


Extracting Stealing:   0%|          | 0/100 [00:00<?, ?vid/s]


📂 Processing Class: Vandalism (50 videos)


Extracting Vandalism:   0%|          | 0/50 [00:00<?, ?vid/s]


✅ PHASE 1 COMPLETE
📄 Metadata saved to: C:\UCF_video_dataset\Processed_Clips\dataset_metadata.json
📊 Total Videos Processed: 1900
Next Step: Load 'dataset_metadata.json' in Phase 2.
